# Imports

In [14]:
!python3 -m pip install matplotlib
!python3 -m pip install scikit-learn
!python3 -m pip install scipy
!python3 -m pip install tensorflow
!python3 -m pip install numpy
!python3 -m pip install pandas
!python3 -m pip install os
!python3 -m pip install setuptools

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [15]:
import tensorflow as tf

from tensorflow.image import resize
from tensorflow.keras.backend import clear_session
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.metrics import  Recall, CategoricalAccuracy
from IPython.display import clear_output

from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import concatenate as concat
from scipy.stats import entropy
import os

np.random.seed(0)
